In [1]:
import os
import pandas as pd
data = pd.read_csv('data/data.csv', header=None)
data

,0,1,2,3
0,300000,«Наёмники блокчейна» — как ими становятся?,"['Dash', 'блокчейн', 'криптовалюта', 'управлен...",data/habr/post__300000.txt
1,300002,Динамичность должна быть безопасной,"['стартапы', 'программы', 'разработка программ...",data/habr/post__300002.txt
2,300004,Издательства выступают против блокировщика рек...,"['mozilla', 'Brave', 'блокировщики рекламы', '...",data/habr/post__300004.txt
3,300006,"Выход из зоны комфорта, как новая бизнес-модель","['бизнес-модель', 'it', 'стартап', 'интернет',...",data/habr/post__300006.txt
4,300008,«Уберизация» поможет российскому рынку рекрути...,"['mail.ru group', 'headhunter', 'российский ры...",data/habr/post__300008.txt
...,...,...,...,...
177,300390,Капитализация Kamcord превысила $100 млн после...,"['Time Warner', 'time warner cable', 'kamcord'...",data/habr/post__300390.txt
178,300392,Конференция ISDEF-2016 в Казани: почему это бы...,"['isdef', 'конференция', 'ит-бизнес', '\n ...",data/habr/post__300392.txt
179,300394,AVO и другие альтернативы роумингу,"['роуминг', 'voip', 'конкуренты', 'стартапы', ...",data/habr/post__300394.txt
180,300396,Шпаргалка в офис: ключевые метрики для развити...,"['метрики', 'развитие бизнеса', '\n ...",data/habr/post__300396.txt


In [2]:
from utils import read_file
publications = [read_file(path) for path in data[3]]
joined_publications = ' '.join(publications)

In [3]:
import matplotlib.pyplot as plt
from wordcloud import  WordCloud

def plot_wordcloud(text, title=''):
    plt.figure(figsize=(8,10))
    wc = WordCloud(max_words=1000, random_state=1).generate(text)
    plt.title(title)
    plt.imshow(wc)
    plt.show()

In [4]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Предобработки
- Удаление пробельных символов
- Удаление спец символов
- Токенезация
- Удаление стандартного списка стоп-слов
- Удаление чисел
- Приведение к нижнему регистру

In [5]:
import re
from utils import compose
def remove_stopwords(stop_words):
    return lambda tokens: (token for token in tokens if token not in stop_words)
def tolower(tokens):
    return (token.lower() for token in tokens)
def tokenize(text):
    return word_tokenize(text, language='russian')   
def remove_numbers(tokens):
    def is_number(string):
        try:
            float(string)
            return True
        except ValueError:
            return False
    return (token for token in tokens if not is_number(token))
def remove_spaces(text):
    return re.sub(r'\s', ' ', text)
def remove_special(special):
    return lambda text: re.sub(special + '+', ' ', text)

## Специальные символы
- Арифметические операции
- Знаки препинания
- Различные скобки
- Слэши
- "Мягкий" дефис: \xad 
- А также $, #, №, @, €

In [6]:
special_symbols = r'''[-+\%*;:\'"()[\]^$#№@!.,?/|\\<>{}—«»€–\xad]'''
stop_words = set(stopwords.words('russian'))
preprocesses = [remove_spaces, remove_special(special_symbols), tokenize, remove_stopwords(stop_words), remove_numbers, tolower]

In [7]:
preprocessed_publications = [list(compose(preprocesses)(publication)) for publication in publications]

In [8]:
preprocessed_publications

го',
  'времени',
  'kamcord',
  'предоставляет',
  'возможность',
  'трансляции',
  'контента',
  'экрана',
  'особенно',
  'популярным',
  'kamcord',
  'оказался',
  'среди',
  'индустрии',
  'мобильных',
  'игр',
  'такие',
  'компании',
  'gameloft',
  'начали',
  'интегрировать',
  'поддержку',
  'возможности',
  'стриминга',
  'собственные',
  'приложения',
  'подобный',
  'контент',
  'ощутимо',
  'повышает',
  'вовлечённость',
  'пользователей',
  'игровой',
  'процесс',
  'также',
  'потенциально',
  'снижает',
  'затраты',
  'привлечение',
  'новых',
  'благодаря',
  'повышенной',
  'популярности',
  'бренда',
  'проекта',
  'сети',
  'одной',
  'причин',
  'интереса',
  'time',
  'warner',
  'kamcord',
  'стали',
  'мобильные',
  'игры',
  'проектов',
  'компании',
  'turner',
  'запускает',
  'трансляции',
  'киберспортивных',
  'мероприятий',
  'телевидении',
  'на',
  'текущий',
  'момент',
  'официальные',
  'партнёры',
  'turner',
  'неизвестны',
  'kamcord',
  'стать',